In [ ]:
"""
圖文選單功能介紹: (圖文選單的上限為1000張)

  用戶能透過點擊選單，進行我方希冀之業務功能。
    
圖文選單製作流程: 
1. 先準備一個json格式的多行字串圖文選單設定檔 (可參考Line Bot Designer的圖文選單)
2. 將多行字串的圖文選單設定檔轉成json物件
3. 將圖文選單的json物件轉換成LINE的RichMenu物件(圖文選單物件)， 然後獲取圖文選單的ID
4. 透過line_bot_api將圖文選單要用的圖片上傳到圖文選單ID，即建置完成


製作好的圖文選單活用: (付費會員跟免費會員綁定不同的選單)
1. 將選單與用戶「做綁定」
2. 檢視用戶目前所綁定的圖文選單
2. 將選單與用戶「解除綁定」
3. 刪除不要的圖文選單選單

"""


In [ ]:
"""
圖文選單設定： (詳細設定參數可參閱Line Bot Designer的圖文選單部份的json程式碼可更快速完成作業)

- 設定圖面大小
- 設定按鍵名稱與功能
    
"""

# 圖文選單設定(多行 => 使用""")
MenuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "區塊鏈自定義選單",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 5,
        "y": 0,
        "width": 824,
        "height": 850
      },
      "action": {
        "type": "message",
        "text": "傳統交易"
      }
    },
    {
      "bounds": {
        "x": 0,
        "y": 850,
        "width": 825,
        "height": 818
      },
      "action": {
        "type": "message",
        "text": "第三方公證人"
      }
    },
    {
      "bounds": {
        "x": 829,
        "y": 5,
        "width": 871,
        "height": 849
      },
      "action": {
        "type": "message",
        "text": "多位公證人"
      }
    },
    {
      "bounds": {
        "x": 825,
        "y": 854,
        "width": 875,
        "height": 814
      },
      "action": {
        "type": "message",
        "text": "多組織多位公證人"
      }
    },
    {
      "bounds": {
        "x": 1700,
        "y": 0,
        "width": 800,
        "height": 858
      },
      "action": {
        "type": "uri",
        "uri": "https://www.binance.com/zh-TW"
      }
    },
    {
      "bounds": {
        "x": 1700,
        "y": 858,
        "width": 800,
        "height": 810
      },
      "action": {
        "type": "message",
        "text": "more"
      }
    }
  ]
}
"""

In [ ]:
"""
安裝套件
"""
!pip install line-bot-sdk

In [ ]:
"""
引用套件: 

用channel_access_token創建line_bot_api，預備用來跟Line進行溝通

"""

from linebot import LineBotApi, WebhookHandler

line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")


In [ ]:
"""

載入上述圖文選單設定檔

並要求line_bot_api將圖文選單上傳至Line
    
"""
import requests
import json

# 1. 載入圖文選單設定檔(json物件)
menuJson = json.loads(MenuRawData)


from linebot.models import RichMenu
# 2. 用line_bot_api的create_rich_menu方法，來創造圖文選單ID
#   => line_bot_api.create_rich_menu(rich_menu=圖文選單物件)，得到的物件是代表圖文選單的ID物件
#         ↓-----------------------------------↑
#   => 圖文選單物件的製作：RichMenu.new_from_json_dict(圖文選單的json設定檔)

""" ***** 重要(困難) ***** """
LineRichMenuID = line_bot_api.create_rich_menu(rich_menu = RichMenu.new_from_json_dict(menuJson))
                    #  *** rich_menu參數 = 圖文選單模組.從json字典新增(json圖文選單設定檔) ***


# 將圖文選單ID印出來看看，然後務必記錄下來(搜索用，或是綁定用戶也須使用)
""" ***** 圖文選單ID要好好保存 ***** """
print(LineRichMenuID)

In [ ]:
"""
透過line_bot_api將事先準備好的圖文選單照片，以Post消息寄發給Line，然後圖文選單就建置完成了

流程：
  - 先載入照片
  - 再要求line_bot_api將圖片傳到指定的圖文選單 (如何指定？ => 圖文選單ID)

"""

# 先載入照片
# 再讓line_bot_api上傳照片，並把生成的圖文選單ID跟要用的圖片綁定
# => line_bot_api.set_rich_menu_image(圖文選單ID, 圖片格式(固定為"image/jpeg"或是"image/png"), 開啟的照片檔案)

with open("/圖文選單示範用圖片.jpg", "rb") as MenuImage:
  SetImageResponse = line_bot_api.set_rich_menu_image(LineRichMenuID, "image/jpeg" ,MenuImage)

  # 打印出來看有沒有成功 (若成功應為空白 => None)
  print(SetImageResponse)


In [ ]:
"""
將做好的圖文選單綁定到特定用戶身上

流程: 
1. 先獲得用戶的ID，然後指派圖文選單給用戶 (透過圖文選單ID來指派不同的選單)
2. 透過line_bot_api告知Line，設定好後，用戶與指派的圖文選單即綁定成功

  *** 偵測用戶ID => event.source.user_id ***

"""

# 讓line_bot_api告知line把圖文選單給指派的user (此以自己做測試) 
# line_bot_api.link_rich_menu_to_user(用戶ID, 要綁定的圖文選單ID)

user_id = "U3741b2fcdd172578b072465a8b6a8fa3"
LinkResult = line_bot_api.link_rich_menu_to_user(user_id, LineRichMenuID)


# 綁定好後，打印出來看看是否成功 (若成功則應該為空白 => None)
print(LinkResult)

In [ ]:
"""
檢視用戶目前所綁定的選單

流程: 

1. 準備用戶ID，並透過line_bot_api來得到並傳回該用戶目前所綁定的圖文選單ID
2. 打印該ID來檢查

"""

# 透過line_bot_api來查詢用戶目前所使用的圖文選單ID (此以自己做測試)
# line_bot_api.get_rich_menu_id_of_user(用戶ID)

user_id = "U3741b2fcdd172578b072465a8b6a8fa3"
user_use_rich_menu_id = line_bot_api.get_rich_menu_id_of_user(user_id) 

# 打印目前用戶所綁定的圖文選單ID
print(user_use_rich_menu_id)

In [ ]:
"""
解除圖文選單與用戶的綁定 (如沒有付費)

流程:
1. 準備用戶ID，並透過line_bot_api來解除指定用戶的圖文選單綁定

"""

# 解除用戶的圖文選單綁定 (此以自己做測試)
# line_bot_api.unlink_rich_menu_from_user(用戶ID)

user_id = "U3741b2fcdd172578b072465a8b6a8fa3"
UnregisterResponse=line_bot_api.unlink_rich_menu_from_user("U3741b2fcdd172578b072465a8b6a8fa3")

# 解除綁定後，打印結果看看是否成功解除綁定 (若成功則應該為空白 => None)
print(UnregisterResponse)

In [ ]:
"""
檢視帳號內，目前有哪些圖文選單
    
流程：
1. 透過line_bot_api，向line查詢我方的圖文選單列表資料
2. 用for迴圈來打印出所有圖文選單ID資訊
"""

# 查詢目前現有的圖文選單數量
# line_bot_api.get_rich_menu_list()

rich_menu_list = line_bot_api.get_rich_menu_list()
for rich_menu in rich_menu_list:
  
  # 藉由rich_menu_id屬性來取得圖文選單ID
  print(rich_menu.rich_menu_id)